## Safety vest detection via retinanet model

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2
import time
import keras
from keras_retinanet import models
from keras_retinanet.utils.image import preprocess_image, resize_image

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
 
class ObjectDetector:
    def __init__(self):
        model_path = './RetinanetModels/PlumsInference.h5' #path to weights
        self.threshold = 0.5
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        self.sess = tf.Session(config=config)
        keras.backend.tensorflow_backend.set_session(self.sess)
        self.model = models.load_model(model_path, backbone_name='resnet50')
        

    def findvest(self, image):
        image = image[:, :, ::-1].copy()
        image = preprocess_image(image)
        image, scale = resize_image(image)

        start_time = time.time()
        boxes, scores, labels = self.model.predict_on_batch(np.expand_dims(image, axis=0))
        boxes/=scale
        print("tf obj_det predict time: ", time.time() - start_time)

        vestboxes = []
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            if score < self.threshold:
                break
            b = np.array(box).astype(int)
            print(b)
            vestboxes.append([label, score, b[0], b[1], b[2], b[3]])
             

        return vestboxes

/home/pasha/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pasha/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pasha/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pasha/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

In [2]:
import cv2
import numpy as np
import sys, os
import time
  

In [5]:
obj_detector = ObjectDetector()

# capture video
if(len(sys.argv) > 1):
    video_path = sys.argv[1]
video_path = "./1.mp4"


try:
    video_capture = cv2.VideoCapture(video_path)
except:
    video_capture = cv2.VideoCapture(0)


ret, frame = video_capture.read()
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out_cap = cv2.VideoWriter(video_path[:-4] + "_res_nobb.avi", fourcc, 20.0, (int(frame.shape[1]), int(frame.shape[0])))

vest_boxes = []               # for storing predictions from object detector

shot_time_start = time.time()
shot_time = time.time()
while True:
    shot_time_start = time.time()

     
    ret, frame = video_capture.read()
    if not ret:
        break

    start_time = time.time()

         
    vest_boxes = obj_detector.findvest(frame)
    for box in vest_boxes:
        xmin = int(box[2])
        ymin = int(box[3])
        xmax = int(box[4])
        ymax = int(box[5])
        cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0))
            
    shot_time = time.time() - shot_time_start
    fps = int(1/shot_time)
    cv2.putText(frame,"FPS: "+str(fps), (30,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,255))
    out_cap.write(frame)
    cv2.imshow('Video', frame)
    print("frame processing time: ", time.time() - start_time)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

out_cap.release()
video_capture.release()
cv2.destroyAllWindows()

tracking <tf.Variable 'Variable_10:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_11:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_12:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_13:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_14:0' shape=(9, 4) dtype=float32> anchors

tf obj_det predict time:  2.031606912612915
frame processing time:  2.0674147605895996
tf obj_det predict time:  0.07135844230651855
frame processing time:  0.10381197929382324
tf obj_det predict time:  0.07287907600402832
frame processing time:  0.10452771186828613
tf obj_det predict time:  0.06378889083862305
frame processing time:  0.08662271499633789
tf obj_det predict time:  0.0613553524017334
frame processing time:  0.0876455307006836
tf obj_det predict time:  0.06153512001037598
frame processing time:  0.08804464340209961
tf obj_det predict time:  0.06191086769104004
frame processing time:  0.08697915077209473
tf o